# Deep Learning Model Training (LSTM)

This notebook trains and evaluates an LSTM model.

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
from sklearn.preprocessing import StandardScaler

# Add src to path
sys.path.append(os.path.abspath('../src'))
from models import ModelFactory, TimeSeriesTrainer
from features import FeatureEngineer

# Load features
data_path = '../data/features/AAPL_1h_features.parquet'

if os.path.exists(data_path):
    df = pd.read_parquet(data_path)
    
    # Prepare sequences
    engineer = FeatureEngineer(df)
    lookback = 30
    X = engineer.create_sequence_features(lookback=lookback)
    
    # Align targets (remove first 'lookback' rows)
    y = df['target_binary'].iloc[lookback:].values
    
    # Split train/val (simple split for DL)
    split_idx = int(len(X) * 0.8)
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_train, y_val = y[:split_idx], y[split_idx:]
    
    # Scale features
    # Note: For 3D array, scale each feature across time/samples
    n_samples, n_timesteps, n_features = X_train.shape
    
    # Reshape to 2D for scaling
    X_train_2d = X_train.reshape(-1, n_features)
    X_val_2d = X_val.reshape(-1, n_features)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_2d).reshape(n_samples, n_timesteps, n_features)
    X_val_scaled = scaler.transform(X_val_2d).reshape(X_val.shape[0], n_timesteps, n_features)
    
    # Build Model
    print(f"Input shape: {X_train_scaled.shape[1:]}")
    model = ModelFactory.get_lstm_model(input_shape=X_train_scaled.shape[1:])
    print(model.summary())
    
    # Train
    trainer = TimeSeriesTrainer(model)
    history = trainer.train_lstm(X_train_scaled, y_train, X_val_scaled, y_val, epochs=20)
    
    # Evaluate
    loss, acc = model.evaluate(X_val_scaled, y_val)
    print(f"Validation Accuracy: {acc:.4f}")
    
    # Save model
    model.save('../models/lstm_model.h5')
    print("Model saved to models/lstm_model.h5")
else:
    print(f"File not found: {data_path}")